In [1]:
import random
from time import time

from matplotlib import pyplot as plt
from mpl_toolkits import axes_grid1
import numpy as np
from IPython.display import Markdown, display
import tensorflow as tf
from tensorflow import keras

import gc
import os

from tabulate import tabulate

# install TF similarity if needed
try:
    import tensorflow_similarity as tfsim  # main package
except ModuleNotFoundError:
    !pip install tensorflow_similarity
    import tensorflow_similarity as tfsim

# INFO messages are not printed.
# This must be run before loading other modules.
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"


tfsim.utils.tf_cap_memory()

# Clear out any old model state.
gc.collect()
tf.keras.backend.clear_session()

print("TensorFlow:", tf.__version__)
print("TensorFlow Similarity:", tfsim.__version__)

In [2]:
from tensorflow.keras import layers
from tensorflow_similarity.layers import MetricEmbedding
from tensorflow_similarity.models import SimilarityModel


# building model
model = tfsim.architectures.EfficientNetSim(
    input_shape= (600, 600, 3), 
    embedding_size = 512, #Max is 512
    variant = "B7",
    weights = "imagenet",
    trainable = "partial", #train last 3 layers
    l2_norm = True,
    include_top = True,
    pooling="max",    # Can change to use `gem` -> GeneralizedMeanPooling2D
#    gem_p=3.0,        # Increase the contrast between activations in the feature map.
)

model.summary()

#ref : https://github.com/tensorflow/similarity/blob/master/api/TFSimilarity/architectures/EfficientNetSim.md

In [3]:
learning_rate = 0.0001
gamma = 256
# init similarity loss
#loss = tfsim.losses.MultiSimilarityLoss()
loss = tfsim.losses.CircleLoss(gamma=gamma)
# compiling and training
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate), 
    loss=loss
)

In [4]:
path = "/kaggle/input/kaprao-horapa-model/Kaprao_Horapa_model.h5"
model.load_weights(path)

In [5]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator()

data = train_datagen.flow_from_directory(
    '/kaggle/input/kaprao-horapa-train-70',
    target_size=(600, 600),
    class_mode='sparse',
    batch_size = 10000,
    subset='training') # set as training data

In [6]:
0# What is indexed
#index_x = validation_generator[0][0][0:5]
#index_y = validation_generator[0][1][0:5]
start_gather = time()
#index_x = data[0][0]
#index_y = data[0][1]
index_data = tf.cast(data[0][0], dtype="int32")  # casted so it can displayed
end_gather = time()

hours_gather, rem_gather = divmod(end_gather - start_gather, 3600)
minutes_gather, seconds_gather = divmod(rem_gather, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours_gather),int(minutes_gather),seconds_gather))

In [7]:
start_index = time()
model.reset_index()
model.index(data[0][0], data[0][1], data=index_data)
end_index = time()

hours_indexing, rem_indexing = divmod(end_index - start_index, 3600)
minutes_indexing, seconds_indexing = divmod(rem_indexing, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours_indexing),int(minutes_indexing),seconds_indexing))

In [8]:
num_examples = 10
num_neigboors = 20

#query image
query_index = random.sample(range(len(data[0][1])), num_examples)
print(query_index)
batch = tf.gather(data[0][0], query_index)
nns = model.lookup(batch, k=num_neigboors)

#num_cols = num_rows = 5
#fig = plt.figure(figsize=(10.0, 10.0))
#grid = axes_grid1.ImageGrid(fig, 111, nrows_ncols=(num_cols, num_rows), axes_pad=0.1)

i = 0
speed = []

for index, nn in zip(query_index, nns):
  i+=1
  display(Markdown("**Query**" + str(i)))
  start_query = time()
  tfsim.visualization.viz_neigbors_imgs(index_data[index], data[0][1][index], nn, cmap="Greys", fig_size=(30, 20))
  end_query = time()

  hours_query, rem_query = divmod(end_query - start_query, 3600)
  minutes_query, seconds_query = divmod(rem_query, 60)
  print("Query " + str(i) + " spends " + "{:0>2}:{:0>2}:{:05.2f}".format(int(hours_query),int(minutes_query),seconds_query))
  
  speed.append(end_query - start_query)

#find average query speed
avg_speed = sum(speed) / len(speed)
print("average speed is " + str(avg_speed) + " seconds")

    # view results close by
#    if index_y[bid] in [1,2]:
#        display(Markdown("**Known Class**"))
#    else:
#        display(Markdown("**Unknown Class**"))
#    tfsim.visualization.viz_neigbors_imgs(index_data[bid], index_y[bid], nn, class_mapping=class_map, cmap="Greys")